In [9]:
import pandas as pdf
from transformers import pipeline

In [2]:
df = pd.read_csv('reviews.csv')

In [31]:
df.head()

,reviewerID,asin,reviewerName,reviewText,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A2FII3I2MBMUIA,B007WTAJTO,1K3,it works as expected. I should have sprung for...,1356220800,23/12/2012,715,0,0
1,A3H99DFEG68SR,B007WTAJTO,1m2,This think has worked out great.Had a diff. br...,1384992000,21/11/2013,382,0,0
2,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"Bought it with Retail Packaging, arrived legit...",1373673600,13/07/2013,513,0,0
3,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,It's mini storage. It doesn't do anything els...,1367193600,29/04/2013,588,0,0
4,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,I have it in my phone and it never skips a bea...,1382140800,19/10/2013,415,0,0


In [12]:
df.shape

(40, 9)

The **pipeline** function is the easiest way to use pre-trained models for common tasks. It handles pre-processing, model inference, and post-processing



```python
from transformers import pipeline

# Example for sentiment analysis
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
result = classifier("I've been waiting for a Hugging Face course my whole life.")
print(result)
```

Directly loading model and tokenizer.
For more control or advanced use cases, you can load the model and its corresponding tokenizer separately. It is recommended to use the Auto* classes for architecture-agnostic loading.



```python
from transformers import AutoTokenizer, AutoModelForSequenceClassificatio
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Example of tokenizing and passing through the model
inputs = tokenizer("Hello, world!", return_tensors="pt")
outputs = model(**inputs)
```





In [ ]:
MODEL = f"distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [47]:
tokenizer.model_max_length = 256

In [53]:
# Example for sentiment analysis
classifier = pipeline("sentiment-analysis",
                      model=MODEL,
                      truncation=True,
                      max_length=512)
result = classifier("I've been waiting for a Hugging Face course my whole life.")
print(result)

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9982948899269104}]


In [54]:
sentiment = classifier(df['reviewText'][0])

In [55]:
sentiment

[{'label': 'NEGATIVE', 'score': 0.9990612864494324}]

In [56]:
sentiment[0]['label']

'NEGATIVE'

In [62]:
df[['sentiment', 'score']] = df['reviewText'].apply(lambda x: pd.Series({
    'sentiment': classifier(x)[0]['label'],
    'score': classifier(x)[0]['score']
    }))

In [65]:
df[['reviewText', 'sentiment', 'score']].head(10)

,reviewText,sentiment,score
0,it works as expected. I should have sprung for...,NEGATIVE,0.999061
1,This think has worked out great.Had a diff. br...,POSITIVE,0.999203
2,"Bought it with Retail Packaging, arrived legit...",POSITIVE,0.649274
3,It's mini storage. It doesn't do anything els...,POSITIVE,0.858108
4,I have it in my phone and it never skips a bea...,POSITIVE,0.930874
5,"It works, but file writes are a bit slower tha...",NEGATIVE,0.990504
6,"I bought 2 of those SanDisk 32 GB microSD , us...",NEGATIVE,0.999605
7,The memory card is an excellent condition and ...,POSITIVE,0.999637
8,I bougth this micro SD card after some trubles...,NEGATIVE,0.987478
9,Ordered this for a Galaxy S3. Lasted a few mo...,NEGATIVE,0.999583


## Outro Jeito

In [ ]:
coluna_de_reviews = df['reviewText']
coluna_de_reviews

In [68]:
from google import genai

In [70]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [71]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
lista_de_analises_de_sentimentos = []

for review_numero, resenha in enumerate(coluna_de_reviews):
    resposta = client.models.generate_content(
        model="gemma-3-27b-it",
        contents=f"""Você irá analisar a resenha que eu te mandarei abaixo, e retorna com uma análise de sentimento.
                    Você deve responder APENAS com uma das seguintes palavras: 'Positiva', 'Negativa' ou 'Neutra',
                    indicando o sentimento relativo àquela resenha específica.

                    Exemplos:
                    'Eu adorei esse produto' -> Positiva
                    'Gostei, mas não é nada de especial' -> Neutra
                    'Odiei esse produto' -> Negativa

                    Segue a resenha a ser analisada: {resenha}
                    """
    )
    lista_de_analises_de_sentimentos.append(resposta.text)
    print(f"Resenha {review_numero}: '{resenha}' -> Sentimento: {resposta.text}")

In [ ]:
df["Análises de Sentimentos"] = lista_de_analises_de_sentimentos